In [452]:
import geopandas as gpd
import pandas as pd

path = "/data/MASTER_dataset.geojson"
gdf = gpd.read_file(path)

print(type(gdf))
print("rows:", len(gdf), "cols:", len(gdf.columns))
gdf.head(2)

<class 'geopandas.geodataframe.GeoDataFrame'>
rows: 1526 cols: 91


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,inland_flood_risk_score,inland_flood_risk_value,inland_flood_risk_rating,hurricane_eal_total,hurricane_eal_score,hurricane_risk_score,hurricane_risk_value,hurricane_risk_rating,in_floodplain,geometry
0,12,011,060303,12011060303,603.03,Census Tract 603.03,G5020,S,1091219,0,...,53.101923,686889.777485,Relatively Moderate,475809.540004,89.452979,92.210038,727508.047604,Relatively High,True,"POLYGON ((-80.22704 26.16284, -80.22704 26.162..."
1,12,086,013600,12086013600,136,Census Tract 136,G5020,S,782295,29373,...,47.346390,615258.127495,Relatively Low,556158.282723,90.580543,93.063405,830655.007813,Relatively High,True,"POLYGON ((-80.33135 25.87703, -80.33079 25.877..."


In [453]:
cols = pd.Series(gdf.columns).sort_values()
cols.to_list()[:30], cols.to_list()[-30:]

(['ALAND',
  'AREA_SQMI',
  'AWATER',
  'COUNTY',
  'COUNTYFP',
  'County FIPS Code',
  'County Name',
  'EP_AGE17',
  'EP_AGE65',
  'EP_CROWD',
  'EP_DISABL',
  'EP_GROUPQ',
  'EP_HBURD',
  'EP_LIMENG',
  'EP_MINRTY',
  'EP_MOBILE',
  'EP_MUNIT',
  'EP_NOHSDP',
  'EP_NOVEH',
  'EP_POV150',
  'EP_SNGPNT',
  'EP_UNEMP',
  'EP_UNINSUR',
  'FUNCSTAT',
  'F_THEME1',
  'F_THEME2',
  'F_THEME3',
  'F_THEME4',
  'F_TOTAL',
  'GEOID'],
 ['hurricane_eal_total',
  'hurricane_risk_rating',
  'hurricane_risk_score',
  'hurricane_risk_value',
  'in_floodplain',
  'inland_flood_eal_score',
  'inland_flood_eal_total',
  'inland_flood_risk_rating',
  'inland_flood_risk_score',
  'inland_flood_risk_value',
  'nri_rating',
  'nri_score',
  'nri_value',
  'pop_2020',
  'resilience_rating',
  'resilience_score',
  'social_vuln_rating',
  'social_vuln_score',
  'state',
  'svi_overall_pctile',
  'svi_overall_score',
  'svi_theme1_pctile',
  'svi_theme1_score',
  'svi_theme2_pctile',
  'svi_theme2_score',
 

In [454]:
rename_map = {
    # IDs / geography
    "GEOID": "geoid",
    "County Name": "county_name",
    "County FIPS Code": "county_fips_code",
    "COUNTYFP": "county_fips",
    "COUNTY": "county_name_alt",
    "state": "state_name_alt",
    "tract": "tract_name_alt",  # we'll replace with a cleaner tract label later if desired

    # TIGER land/water
    "ALAND": "land_area_m2",
    "AWATER": "water_area_m2",
    "AREA_SQMI": "area_sqmi",

    # ACS

    # SVI percentiles/scores
    "svi_theme1_pctile": "svi_ses_pctile",
    "svi_theme2_pctile": "svi_household_comp_pctile",
    "svi_theme3_pctile": "svi_minority_lang_pctile",
    "svi_theme4_pctile": "svi_housing_transport_pctile",
    "svi_overall_pctile": "svi_overall_pctile",

    "svi_theme1_score": "svi_ses_score",
    "svi_theme2_score": "svi_household_comp_score",
    "svi_theme3_score": "svi_minority_lang_score",
    "svi_theme4_score": "svi_housing_transport_score",
    "svi_overall_score": "svi_overall_score",

    # SVI EP_* (percent values 0–100)
    "EP_POV150": "svi_poverty_150pct_pct",
    "EP_UNEMP": "svi_unemployed_pct",
    "EP_HBURD": "svi_housing_cost_burden_pct",
    "EP_NOHSDP": "svi_no_hs_diploma_pct",
    "EP_UNINSUR": "svi_uninsured_pct",
    "EP_AGE65": "svi_age_65plus_pct",
    "EP_AGE17": "svi_age_under17_pct",
    "EP_DISABL": "svi_disability_pct",
    "EP_SNGPNT": "svi_single_parent_hh_pct",
    "EP_LIMENG": "svi_limited_english_pct",
    "EP_MINRTY": "svi_minority_pct",
    "EP_MUNIT": "svi_multiunit_housing_pct",
    "EP_MOBILE": "svi_mobile_homes_pct",
    "EP_CROWD": "svi_crowding_pct",
    "EP_NOVEH": "svi_no_vehicle_pct",
    "EP_GROUPQ": "svi_group_quarters_pct",

    # SVI flags
    "F_THEME1": "svi_flag_theme1",
    "F_THEME2": "svi_flag_theme2",
    "F_THEME3": "svi_flag_theme3",
    "F_THEME4": "svi_flag_theme4",
    "F_TOTAL": "svi_flag_total",

    # NRI core
    "pop_2020": "nri_population_2020",
    "nri_value": "nri_composite_value",
    "nri_score": "nri_composite_score",
    "nri_rating": "nri_composite_rating",
    "social_vuln_score": "nri_social_vuln_score",
    "social_vuln_rating": "nri_social_vuln_rating",
    "resilience_score": "nri_resilience_score",
    "resilience_rating": "nri_resilience_rating",

    # NRI hazard-specific + losses
    "coastal_flood_eal_total": "nri_coastal_flood_eal_total_usd",
    "coastal_flood_eal_score": "nri_coastal_flood_eal_score",
    "coastal_flood_risk_score": "nri_coastal_flood_risk_score",
    "coastal_flood_risk_value": "nri_coastal_flood_risk_value",
    "coastal_flood_risk_rating": "nri_coastal_flood_risk_rating",

    "inland_flood_eal_total": "nri_inland_flood_eal_total_usd",
    "inland_flood_eal_score": "nri_inland_flood_eal_score",
    "inland_flood_risk_score": "nri_inland_flood_risk_score",
    "inland_flood_risk_value": "nri_inland_flood_risk_value",
    "inland_flood_risk_rating": "nri_inland_flood_risk_rating",

    "hurricane_eal_total": "nri_hurricane_eal_total_usd",
    "hurricane_eal_score": "nri_hurricane_eal_score",
    "hurricane_risk_score": "nri_hurricane_risk_score",
    "hurricane_risk_value": "nri_hurricane_risk_value",
    "hurricane_risk_rating": "nri_hurricane_risk_rating",

    # NFHL
    "in_floodplain": "nfhl_in_floodplain",

    # TIGER internal status (usually not needed)
    "FUNCSTAT": "tiger_funcstat"
}

gdf = gdf.rename(columns={k: v for k, v in rename_map.items() if k in gdf.columns})
gdf.columns[:25], gdf.columns[-10:]

(Index(['STATEFP', 'county_fips', 'TRACTCE', 'geoid', 'NAME', 'NAMELSAD',
        'MTFCC', 'tiger_funcstat', 'land_area_m2', 'water_area_m2', 'INTPTLAT',
        'INTPTLON', 'acs_median_hh_income', 'acs_population', 'acs_households',
        'acs_avg_hh_size', 'state_name_alt', 'county', 'tract_name_alt',
        'county_name', 'ST', 'STATE', 'ST_ABBR', 'county_name_alt', 'STCNTY'],
       dtype='object'),
 Index(['nri_inland_flood_risk_score', 'nri_inland_flood_risk_value',
        'nri_inland_flood_risk_rating', 'nri_hurricane_eal_total_usd',
        'nri_hurricane_eal_score', 'nri_hurricane_risk_score',
        'nri_hurricane_risk_value', 'nri_hurricane_risk_rating',
        'nfhl_in_floodplain', 'geometry'],
       dtype='object'))

In [455]:
gdf["geoid"] = gdf["geoid"].astype(str).str.replace(r"\.0$", "", regex=True).str.zfill(11)

# booleans
if "nfhl_in_floodplain" in gdf.columns:
    gdf["nfhl_in_floodplain"] = gdf["nfhl_in_floodplain"].astype("boolean")

# numeric coercion for key fields (Tableau-friendly)
num_cols = [
    # ACS demographics
    "acs_population",
    "acs_households",
    "acs_median_hh_income",
    "acs_avg_hh_size",

    # Social vulnerability
    "svi_overall_pctile",
    "svi_overall_score",

    # National Risk Index (composite)
    "nri_composite_score",
    "nri_composite_value",

    # Hazard-specific losses
    "nri_coastal_flood_eal_total_usd",
    "nri_inland_flood_eal_total_usd",
    "nri_hurricane_eal_total_usd",
]

for c in [c for c in num_cols if c in gdf.columns]:
    gdf[c] = pd.to_numeric(gdf[c], errors="coerce")

gdf[["geoid", "acs_population", "acs_median_hh_income", "svi_overall_pctile", "nri_composite_score"]].head()


,geoid,acs_population,acs_median_hh_income,svi_overall_pctile,nri_composite_score
0,12011060303,6883.0,38319.0,0.9742,58.201039
1,12086013600,5857.0,44688.0,0.8396,57.221172
2,12086013700,6199.0,41081.0,0.9560,74.054915
3,12086980100,NaN,NaN,NaN,0.083241
4,12011110501,3996.0,78599.0,0.5381,10.904594


In [456]:
gdf[[
    "acs_population",
    "acs_households",
    "acs_median_hh_income",
    "acs_avg_hh_size"
]].isna().mean()

acs_population          0.013106
acs_households          0.015727
acs_median_hh_income    0.022936
acs_avg_hh_size         0.015727
dtype: float64

In [457]:
drop_dupes = ["county_name_alt", "state_name_alt", "tract_name_alt"]
gdf = gdf.drop(columns=[c for c in drop_dupes if c in gdf.columns], errors="ignore")

In [458]:
# Recreate land / water classification BEFORE dropping TIGER fields

gdf["total_area_m2"] = gdf["land_area_m2"] + gdf["water_area_m2"]

gdf["land_share"] = np.where(
    gdf["total_area_m2"] > 0,
    gdf["land_area_m2"] / gdf["total_area_m2"],
    np.nan
)

def classify_tract_land_type(land_share):
    if pd.isna(land_share):
        return "unknown"
    elif land_share < 0.05:
        return "water"
    elif land_share < 0.25:
        return "mostly_water"
    else:
        return "land"

gdf["tract_land_type"] = gdf["land_share"].apply(classify_tract_land_type)

In [459]:
# Explicit TIGER/internal fields to drop
tiger_drop = [
    "tiger_funcstat",
    "land_area_m2",
    "water_area_m2",
    "area_sqmi",
    "state_fips",
    "county_fips",
    "tract_code",
    "county_fips_code",
]

gdf = gdf.drop(columns=[c for c in tiger_drop if c in gdf.columns], errors="ignore")

raw_tiger_like = [
    "ALAND", "AWATER", "AREA_SQMI", "FUNCSTAT",
    "STATEFP", "COUNTYFP", "TRACTCE", "MTFCC", "NAMELSAD", "NAME"
]

gdf = gdf.drop(columns=[c for c in raw_tiger_like if c in gdf.columns], errors="ignore")

print("Cols after dropping TIGER internals:", len(gdf.columns))

Cols after dropping TIGER internals: 79


In [460]:
gdf["tract_land_type"].value_counts(normalize=True).round(3) * 100

tract_land_type
land            97.9
mostly_water     1.6
water            0.5
Name: proportion, dtype: float64

In [461]:
front = [
    "geoid",
    "county_name",
    "acs_population",
    "acs_households",
    "acs_median_hh_income",
    "acs_avg_hh_size",
    "svi_overall_pctile",
    "svi_overall_score",
    "nri_composite_score",
    "nri_composite_value",
    "nri_composite_rating",
    "nri_eal_total_usd",     # if present
    "nfhl_in_floodplain",
    "geometry",
]
front = [c for c in front if c in gdf.columns]
rest = [c for c in gdf.columns if c not in front]
gdf = gdf[front + rest]

gdf.head(2)

,geoid,county_name,county_name,acs_population,acs_households,acs_median_hh_income,acs_avg_hh_size,svi_overall_pctile,svi_overall_score,nri_composite_score,...,nri_inland_flood_risk_value,nri_inland_flood_risk_rating,nri_hurricane_eal_total_usd,nri_hurricane_eal_score,nri_hurricane_risk_score,nri_hurricane_risk_value,nri_hurricane_risk_rating,total_area_m2,land_share,tract_land_type
0,12011060303,Broward,Broward,6883.0,2366.0,38319.0,2.81,0.9742,11.8481,58.201039,...,686889.777485,Relatively Moderate,475809.540004,89.452979,92.210038,727508.047604,Relatively High,1091219,1.000000,land
1,12086013600,Miami-Dade,Miami-Dade,5857.0,2011.0,44688.0,2.91,0.8396,10.0686,57.221172,...,615258.127495,Relatively Low,556158.282723,90.580543,93.063405,830655.007813,Relatively High,811668,0.963812,land


In [462]:
[g for g in gdf.columns if "tiger" in g.lower() or g.lower() in {"aland","awater","statefp","countyfp","tractce","mtfcc","funcstat"}]

[]

In [463]:
# Duplicate column names (exact)
pd.Series(gdf.columns).value_counts().loc[lambda x: x > 1]


county_name    2
Name: count, dtype: int64

In [464]:
from itertools import combinations

duplicate_cols = []

for col1, col2 in combinations(gdf.columns, 2):
    # Skip geometry
    if col1 == "geometry" or col2 == "geometry":
        continue

    try:
        if gdf[col1].equals(gdf[col2]):
            duplicate_cols.append((col1, col2))
    except Exception:
        pass

duplicate_cols[:10], len(duplicate_cols)


([('county_name', 'county_name')], 1)

In [465]:
dup_df = pd.DataFrame(duplicate_cols, columns=["column_1", "column_2"])
dup_df

,column_1,column_2
0,county_name,county_name


In [466]:
# Where are the duplicate column names?
dup_name = "county_name"
idxs = [i for i, c in enumerate(gdf.columns) if c == dup_name]
idxs

[1, 2]

In [467]:
c1, c2 = idxs[0], idxs[1]

same = gdf.iloc[:, c1].equals(gdf.iloc[:, c2])
print("Are the two county_name columns identical?", same)

# Peek a few values side-by-side
pd.DataFrame({
    "county_name_1": gdf.iloc[:, c1].head(10),
    "county_name_2": gdf.iloc[:, c2].head(10)
})

Are the two county_name columns identical? False


,county_name_1,county_name_2
0,Broward,Broward
1,Miami-Dade,Miami-Dade
2,Miami-Dade,Miami-Dade
3,Miami-Dade,Miami-Dade
4,Broward,Broward
5,Miami-Dade,Miami-Dade
6,Miami-Dade,Miami-Dade
7,Palm Beach,Palm Beach
8,Palm Beach,Palm Beach
9,Miami-Dade,Miami-Dade


In [468]:
# Safer way: drop by position using iloc selection:
keep_cols = [i for i in range(len(gdf.columns)) if i != c2]
gdf = gdf.iloc[:, keep_cols]

# Verify
pd.Series(gdf.columns).value_counts().loc[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [469]:
# Keep first occurrence of each column name
gdf = gdf.loc[:, ~pd.Index(gdf.columns).duplicated(keep="first")]

# Verify no duplicate names
pd.Series(gdf.columns).value_counts().loc[lambda x: x > 1]

Series([], Name: count, dtype: int64)

In [470]:
from itertools import combinations

duplicate_value_pairs = []
for a, b in combinations([c for c in gdf.columns if c != "geometry"], 2):
    if gdf[a].equals(gdf[b]):
        duplicate_value_pairs.append((a, b))

duplicate_value_pairs[:20], len(duplicate_value_pairs)


([], 0)

In [471]:
gdf.head(2)

,geoid,county_name,acs_population,acs_households,acs_median_hh_income,acs_avg_hh_size,svi_overall_pctile,svi_overall_score,nri_composite_score,nri_composite_value,...,nri_inland_flood_risk_value,nri_inland_flood_risk_rating,nri_hurricane_eal_total_usd,nri_hurricane_eal_score,nri_hurricane_risk_score,nri_hurricane_risk_value,nri_hurricane_risk_rating,total_area_m2,land_share,tract_land_type
0,12011060303,Broward,6883.0,2366.0,38319.0,2.81,0.9742,11.8481,58.201039,1.777514e+06,...,686889.777485,Relatively Moderate,475809.540004,89.452979,92.210038,727508.047604,Relatively High,1091219,1.000000,land
1,12086013600,Miami-Dade,5857.0,2011.0,44688.0,2.91,0.8396,10.0686,57.221172,1.745034e+06,...,615258.127495,Relatively Low,556158.282723,90.580543,93.063405,830655.007813,Relatively High,811668,0.963812,land


In [472]:
gdf = gdf.loc[:, ~pd.Index(gdf.columns).duplicated(keep="first")]

In [473]:
gdf["county_name"] = (
    gdf["county_name"]
      .astype(str)
      .str.replace(r"\s+County$", "", regex=True)   # remove trailing " County"
      .str.strip()
)

# Optional: title case (keeps Miami-Dade clean)
gdf["county_name"] = gdf["county_name"].replace({
    "Miami-dade": "Miami-Dade"
})

In [474]:
[c for c in gdf.columns if "pop" in c.lower()]

['acs_population', 'nri_population_2020']

In [475]:
drop_pop = ["nri_population_2020", "pop_2020"]
gdf = gdf.drop(columns=[c for c in drop_pop if c in gdf.columns], errors="ignore")

In [476]:
checks = {}

# SVI percentiles should be 0–1
if "svi_overall_pctile" in gdf.columns:
    checks["svi_overall_pctile_out_of_range"] = int(((gdf["svi_overall_pctile"] < 0) | (gdf["svi_overall_pctile"] > 1)).sum())

# EP_* percent fields should be 0–100
ep_cols = [c for c in gdf.columns if c.startswith("svi_") and c.endswith("_pct")]
if ep_cols:
    bad = 0
    for c in ep_cols:
        bad += int(((gdf[c] < 0) | (gdf[c] > 100)).sum())
    checks["svi_pct_fields_out_of_range_total_cells"] = bad

checks

{'svi_overall_pctile_out_of_range': 0,
 'svi_pct_fields_out_of_range_total_cells': 0}

In [477]:
id_cols = ["geoid", "county_name"]
acs_cols = [c for c in gdf.columns if c.startswith("acs_")]
svi_core = [
    "svi_overall_pctile",
    "svi_ses_pctile", "svi_household_comp_pctile", "svi_minority_lang_pctile", "svi_housing_transport_pctile",
    "svi_overall_score",
    "svi_ses_score", "svi_household_comp_score", "svi_minority_lang_score", "svi_housing_transport_score",
]
svi_core = [c for c in svi_core if c in gdf.columns]

svi_detail = [c for c in gdf.columns if c.startswith("svi_") and c.endswith("_pct")]
svi_flags = [c for c in gdf.columns if c.startswith("svi_flag_")]

nri_core = [c for c in [
    "nri_composite_score", "nri_composite_value", "nri_composite_rating",
    "nri_eal_total_usd", "nri_eal_score", "nri_eal_rating",
    "nri_social_vuln_score", "nri_social_vuln_rating",
    "nri_resilience_score", "nri_resilience_rating",
] if c in gdf.columns]

nri_hazards = [c for c in gdf.columns if c.startswith("nri_") and any(k in c for k in ["coastal_flood", "inland_flood", "hurricane"])]

exposure_cols = [c for c in ["nfhl_in_floodplain"] if c in gdf.columns]

# Keep geometry last for readability
geom = ["geometry"]

ordered = []
for block in [id_cols, acs_cols, svi_core, svi_detail, svi_flags, nri_core, nri_hazards, exposure_cols, geom]:
    for c in block:
        if c in gdf.columns and c not in ordered:
            ordered.append(c)

# Add any remaining columns not captured (just in case)
ordered += [c for c in gdf.columns if c not in ordered]

gdf = gdf[ordered]
gdf.columns[:25]

Index(['geoid', 'county_name', 'acs_population', 'acs_households',
       'acs_median_hh_income', 'acs_avg_hh_size', 'svi_overall_pctile',
       'svi_ses_pctile', 'svi_household_comp_pctile',
       'svi_minority_lang_pctile', 'svi_housing_transport_pctile',
       'svi_overall_score', 'svi_ses_score', 'svi_household_comp_score',
       'svi_minority_lang_score', 'svi_housing_transport_score',
       'svi_poverty_150pct_pct', 'svi_unemployed_pct',
       'svi_housing_cost_burden_pct', 'svi_no_hs_diploma_pct',
       'svi_uninsured_pct', 'svi_age_65plus_pct', 'svi_age_under17_pct',
       'svi_disability_pct', 'svi_single_parent_hh_pct'],
      dtype='object')

In [478]:
[c for c in gdf.columns if c.startswith("nri_")][:40]

['nri_composite_score',
 'nri_composite_value',
 'nri_composite_rating',
 'nri_social_vuln_score',
 'nri_social_vuln_rating',
 'nri_resilience_score',
 'nri_resilience_rating',
 'nri_coastal_flood_eal_total_usd',
 'nri_coastal_flood_eal_score',
 'nri_coastal_flood_risk_score',
 'nri_coastal_flood_risk_value',
 'nri_coastal_flood_risk_rating',
 'nri_inland_flood_eal_total_usd',
 'nri_inland_flood_eal_score',
 'nri_inland_flood_risk_score',
 'nri_inland_flood_risk_value',
 'nri_inland_flood_risk_rating',
 'nri_hurricane_eal_total_usd',
 'nri_hurricane_eal_score',
 'nri_hurricane_risk_score',
 'nri_hurricane_risk_value',
 'nri_hurricane_risk_rating']

In [479]:
# Ensure EAL totals are numeric + non-negative
import pandas as pd
import numpy as np

eal_cols = [
    "nri_coastal_flood_eal_total_usd",
    "nri_inland_flood_eal_total_usd",
    "nri_hurricane_eal_total_usd",
]

for c in eal_cols:
    if c in gdf.columns:
        gdf[c] = pd.to_numeric(gdf[c], errors="coerce")
        gdf.loc[gdf[c] < 0, c] = np.nan  # defensively remove negatives

In [480]:
# Create a combined “flood EAL” and “top hazard EAL”
gdf["nri_flood_eal_total_usd"] = (
    gdf["nri_coastal_flood_eal_total_usd"].fillna(0) +
    gdf["nri_inland_flood_eal_total_usd"].fillna(0)
)

gdf["nri_top3_hazards_eal_total_usd"] = (
    gdf["nri_flood_eal_total_usd"].fillna(0) +
    gdf["nri_hurricane_eal_total_usd"].fillna(0)
)

In [481]:
# Per-capita EAL (individual exposure)
gdf["acs_population"] = pd.to_numeric(gdf["acs_population"], errors="coerce")

gdf["nri_top3_eal_per_capita_usd"] = np.where(
    gdf["acs_population"] > 0,
    gdf["nri_top3_hazards_eal_total_usd"] / gdf["acs_population"],
    np.nan
)

In [482]:
# Per household EAL
gdf["nri_top3_eal_per_household_usd"] = np.where(
    gdf["acs_households"] > 0,
    gdf["nri_top3_hazards_eal_total_usd"] / gdf["acs_households"],
    np.nan
)

In [483]:
# EAL relative to income (another wealth-risk proxy)
gdf["acs_median_hh_income"] = pd.to_numeric(gdf["acs_median_hh_income"], errors="coerce")

In [484]:
# Quick sanity summaries
summary_cols = [
    "nri_flood_eal_total_usd",
    "nri_top3_hazards_eal_total_usd",
    "nri_top3_eal_per_capita_usd",
    "nri_top3_eal_per_household_usd"
]

gdf[summary_cols].describe(include="all")

,nri_flood_eal_total_usd,nri_top3_hazards_eal_total_usd,nri_top3_eal_per_capita_usd,nri_top3_eal_per_household_usd
count,1.526000e+03,1.526000e+03,1506.000000,1502.000000
mean,5.857778e+05,1.216847e+06,578.107204,1268.699622
std,9.986592e+05,1.477035e+06,9377.017106,12153.355343
min,0.000000e+00,0.000000e+00,0.000000,147.718736
25%,2.631862e+05,6.175668e+05,170.088998,459.847721
50%,4.286414e+05,9.322059e+05,240.542686,630.550920
75%,6.931392e+05,1.429525e+06,360.342436,950.312423
max,3.312900e+07,4.166902e+07,363554.563643,458668.292042


In [485]:
gdf[["nri_top3_eal_per_capita_usd"]].describe()

,nri_top3_eal_per_capita_usd
count,1506.000000
mean,578.107204
std,9377.017106
min,0.000000
25%,170.088998
50%,240.542686
75%,360.342436
max,363554.563643


In [486]:
gdf[["nri_top3_eal_per_household_usd"]].describe()

,nri_top3_eal_per_household_usd
count,1502.000000
mean,1268.699622
std,12153.355343
min,147.718736
25%,459.847721
50%,630.550920
75%,950.312423
max,458668.292042


In [487]:
# Create capped (winsorized) per-capita EAL
cap = gdf["nri_top3_eal_per_capita_usd"].quantile(0.99)

gdf["nri_top3_eal_per_capita_usd_capped"] = gdf["nri_top3_eal_per_capita_usd"].clip(upper=cap)

cap

np.float64(1652.54985056954)

In [488]:
# Create capped (winsorized) household EAL
cap_hh = gdf["nri_top3_eal_per_household_usd"].quantile(0.99)

gdf["nri_top3_eal_per_household_usd_capped"] = (
    gdf["nri_top3_eal_per_household_usd"].clip(upper=cap_hh)
)

cap_hh

np.float64(4735.345874719349)

In [489]:
# Create log-scaled version (for maps & rankings)
gdf["nri_top3_eal_per_capita_log"] = np.log1p(
    gdf["nri_top3_eal_per_capita_usd"]
)

gdf["nri_top3_eal_per_household_log"] = np.log1p(
    gdf["nri_top3_eal_per_household_usd"]
)

In [490]:
# Add a clean categorical risk tier (policy-ready)
gdf["eal_per_capita_risk_tier"] = pd.qcut(
    gdf["nri_top3_eal_per_capita_usd"],
    q=[0, 0.25, 0.5, 0.75, 0.9, 1.0],
    labels=["Low", "Moderate", "High", "Very High", "Extreme"],
    duplicates="drop"
)

gdf["eal_per_household_risk_tier"] = pd.qcut(
    gdf["nri_top3_eal_per_household_usd"],
    q=[0, 0.25, 0.5, 0.75, 0.9, 1.0],
    labels=["Low", "Moderate", "High", "Very High", "Extreme"],
    duplicates="drop"
)

In [491]:
# Sanity check
[c for c in gdf.columns if "eal" in c.lower() or "risk_tier" in c.lower()]

['nri_coastal_flood_eal_total_usd',
 'nri_coastal_flood_eal_score',
 'nri_inland_flood_eal_total_usd',
 'nri_inland_flood_eal_score',
 'nri_hurricane_eal_total_usd',
 'nri_hurricane_eal_score',
 'eal_total',
 'eal_score',
 'eal_rating',
 'nri_flood_eal_total_usd',
 'nri_top3_hazards_eal_total_usd',
 'nri_top3_eal_per_capita_usd',
 'nri_top3_eal_per_household_usd',
 'nri_top3_eal_per_capita_usd_capped',
 'nri_top3_eal_per_household_usd_capped',
 'nri_top3_eal_per_capita_log',
 'nri_top3_eal_per_household_log',
 'eal_per_capita_risk_tier',
 'eal_per_household_risk_tier']

In [492]:
[c for c in gdf.columns if "eal" in c.lower() and "total" in c.lower()]

['nri_coastal_flood_eal_total_usd',
 'nri_inland_flood_eal_total_usd',
 'nri_hurricane_eal_total_usd',
 'eal_total',
 'nri_flood_eal_total_usd',
 'nri_top3_hazards_eal_total_usd']

In [493]:
# Rename composite EAL to a clear baseline name
gdf = gdf.rename(columns={"eal_total": "nri_all_hazards_eal_total_usd"})
gdf["nri_all_hazards_eal_total_usd"] = pd.to_numeric(gdf["nri_all_hazards_eal_total_usd"], errors="coerce")
gdf.loc[gdf["nri_all_hazards_eal_total_usd"] < 0, "nri_all_hazards_eal_total_usd"] = np.nan

In [494]:
# Add “flood + hurricane share of total EAL”
gdf["flood_hurricane_share_of_total_eal"] = np.where(
    gdf["nri_all_hazards_eal_total_usd"] > 0,
    gdf["nri_top3_hazards_eal_total_usd"] / gdf["nri_all_hazards_eal_total_usd"],
    np.nan
)

In [495]:
# Add a policy-friendly dominance category
gdf["flood_hurricane_dominance"] = pd.cut(
    gdf["flood_hurricane_share_of_total_eal"],
    bins=[0, 0.25, 0.5, 0.75, 1.01],
    labels=["Low", "Moderate", "High", "Very High"]
)

In [496]:
gdf["flood_hurricane_share_of_total_eal"].describe()

count    1520.000000
mean        0.877270
std         0.059283
min         0.194690
25%         0.841378
50%         0.883157
75%         0.919118
max         0.992635
Name: flood_hurricane_share_of_total_eal, dtype: float64

In [497]:
gdf["flood_hurricane_dominance"].value_counts(dropna=False)

flood_hurricane_dominance
Very High    1500
High           17
NaN             6
Moderate        2
Low             1
Name: count, dtype: int64

In [498]:
gdf["nri_all_hazards_eal_per_capita_usd"] = np.where(
    gdf["acs_population"] > 0,
    gdf["nri_all_hazards_eal_total_usd"] / gdf["acs_population"],
    np.nan
)

gdf["nri_all_hazards_eal_per_household_usd"] = np.where(
    gdf["acs_population"] > 0,
    gdf["nri_all_hazards_eal_total_usd"] / gdf["acs_households"],
    np.nan
)

In [499]:
gdf["flood_hurricane_share_of_total_eal"].describe()

count    1520.000000
mean        0.877270
std         0.059283
min         0.194690
25%         0.841378
50%         0.883157
75%         0.919118
max         0.992635
Name: flood_hurricane_share_of_total_eal, dtype: float64

In [500]:
gdf.head(2)

,geoid,county_name,acs_population,acs_households,acs_median_hh_income,acs_avg_hh_size,svi_overall_pctile,svi_ses_pctile,svi_household_comp_pctile,svi_minority_lang_pctile,...,nri_top3_eal_per_capita_usd_capped,nri_top3_eal_per_household_usd_capped,nri_top3_eal_per_capita_log,nri_top3_eal_per_household_log,eal_per_capita_risk_tier,eal_per_household_risk_tier,flood_hurricane_share_of_total_eal,flood_hurricane_dominance,nri_all_hazards_eal_per_capita_usd,nri_all_hazards_eal_per_household_usd
0,12011060303,Broward,6883.0,2366.0,38319.0,2.81,0.9742,0.9430,0.8325,0.9215,...,134.396870,390.977876,4.908210,5.971205,Low,Low,0.795717,Very High,168.900362,491.352998
1,12086013600,Miami-Dade,5857.0,2011.0,44688.0,2.91,0.8396,0.9022,0.6404,0.9805,...,165.360753,481.610110,5.114159,6.179209,Low,Moderate,0.828946,Very High,199.483251,580.991249


In [501]:
gdf.to_file("/data/MASTER_dataset CLEANED.geojson", driver="GeoJSON")
gdf.drop(columns="geometry").to_csv("/data/MASTER_dataset CLEANED.csv", index=False)
gdf.to_parquet("/data/MASTER_dataset CLEANED.parquet", index=False)